 \ \ \ \ \ \  **WARNING!** \ \ \ \ \ \
 THE FOLLOWING CODE CONTAINS TOXIC SPEECH AND HARMFUL CONTENTS.

Welcome to the Un4CHANate Analysis notebook II ! In which Sentiment analysis will be performed! Now that you have gained a detailed knowledge about the densest groups of topics in your dataset, and added them as a third column, it is time to analyse their average sentiment. In this second analysis notebook, we will pursue a sentiment analysis of the ten densest groups of topics by using a fine-tuned DistilBERT model from HuggingFace. Our main scope, here, is to create and append **three more columns to our dataset: "positive sentiment", "negative sentiment" and an overall sentiment score**.

**STEP I:**

Replace file_path with csv created in DATA_ANALYSIS_4CHAN_II or use the second demo file called, 'demo_politically_incorrect_7d_jan_6_2021_topics.csv'. 

In [ ]:
import pandas as pd

file_path = r''# Replace with csv created in DATA_ANALYSIS_4CHAN_II

try:
  test_dataset = pd.read_csv(file_path)
  print(df.head()) # Display the first few rows of the DataFrame
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
except pd.errors.ParserError:
  print(f"Error: Could not parse the CSV file at {file_path}. Check the file format.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

**STEP II:**

Run cell below. 

In [ ]:
df = test_dataset.copy() #always make a copy, so that the original dataset does not get modified!

**STEP III:**

Let's get to our sentiment analysis, shall we?

Run the cells below in order to display your results.

Make sure to pip install
- !pip install torch
- !pip install transformers
- !pip install matplotlib
- !pip install seaborn


In [ ]:
allowed_topics_2 = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
df_2 = df[df['topics'].isin(allowed_topics_2)]

In [ ]:
print(df_2.info())

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification # Import DistilBertTokenizer and DistilBertForSequenceClassification
import pandas as pd
# After having installed and imported all the necessary libraries and tools,we can proceed with the sentiment analysis.
# As it can be observed below, a DistilBERT model from the Huggingface library is directly imported, as both the actual model that
#is going to perform the analysis, and as atokenizer, that can make the comments'text data machine-readable. Such a code isalways available on the HuggingFace 
# model's website as well, ready to be copied and pasted.
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english") 
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# the aim ofour analysis here is not just that  of having an overall sentiment score of the whole dataset,
# but that of having a sentiment score for every single comment. We can do that by iterating the analysis
# over every comment.
if 'comment' in df_2.columns:  
    sentiment_scores = [] 
    for comment in df_2['comment']: # this loop allows us to have a score for every single comment.
        try:
            inputs = tokenizer(comment, return_tensors="pt", truncation=True, max_length=512)
            with torch.no_grad():
                outputs = model(**inputs)
            probabilities = torch.softmax(outputs.logits, dim=1)
            sentiment_scores.append(probabilities.tolist()[0]) # this line appends the score to the comment
        except Exception as e:
            print(f"Error processing comment: {comment[:50]}... Error: {e}")
            sentiment_scores.append([0,0])

    df_2['sentiment_scores'] = sentiment_scores # we now have a column with all the sentiment scores for every comment.
    print(df_2.head()) # Let's see how it looks!
else:
    print("Error: 'comment' column not found in the DataFrame.")

In [ ]:
print(df_2.sample(50))
print(df_2.info())

In [ ]:
# A few problems can be observed. Firstly, if the data type of the 'time' column is not appropriate, we must convert it.
df_2['time'] = pd.to_datetime(df_2['time'])

In [ ]:
# Another problem can be observed in how the model's output reads. It in fact shows two values(negativity and positivity)
#,that cannot be used both to visualize the scores timewise.We therefore need to 1) divide the two values into distinct columnsand 
# 2) determine a threshold to be then used to approximate the values into one overall score. 
threshold = 0.5

df_2['sentiment_score'] = df_2['sentiment_scores'].apply(lambda probs: 1 if probs[1] > threshold else -1)

In [ ]:
print(df_2.sample(5))

In [ ]:
# a column named 'unnamed: 0' keeps showing up.Let's put an end to this irrelevant and utterly pitiful being.
if 'Unnamed: 0' in df_2.columns:
    df_2 = df_2.drop('Unnamed: 0', axis=1)
print(df_2.info())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

positive_probs = df_2['sentiment_scores'].apply(lambda x: x[1])

# Plot the distribution
plt.figure(figsize=(10, 6))
sns.histplot(positive_probs, bins=4, kde=True)
plt.xlabel("Positive Sentiment Probability")
plt.ylabel("Frequency")
plt.title("Distribution of Positive Sentiment Probabilities")
plt.show()

# Explore percentiles:
for percentile in [25, 50, 75]:
    threshold = positive_probs.quantile(percentile / 100)
    print(f"{percentile}th percentile threshold: {threshold}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

daily_sentiment = df_2.groupby(['topics', pd.Grouper(key='time', freq='D')])['sentiment_score'].mean().reset_index()


plt.figure(figsize=(16, 6))
sns.lineplot(x='time', y='sentiment_score', hue='topics', data=daily_sentiment, marker='o')
plt.xlabel('Date')
plt.ylabel('Average Sentiment Score')
plt.title('Average Daily Sentiment Score per Topic')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Topic')
plt.grid(True)
plt.tight_layout()
plt.show()


daily_comments = df_2.groupby(['topics', pd.Grouper(key='time', freq='D')])['comment'].count().reset_index()


plt.figure(figsize=(16, 6))
sns.lineplot(x='time', y='comment', hue='topics', data=daily_comments, marker='o')
plt.xlabel('Date')
plt.ylabel('Number of Comments')
plt.title('Number of Comments per Day per Topic')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Topic')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(16, 10))


plt.subplot(2, 1, 1)
sns.lineplot(x='time', y='sentiment_score', hue='topics', data=daily_sentiment, marker='o')
plt.xlabel('Date')
plt.ylabel('Average Sentiment Score')
plt.title('Average Sentiment Score Over Time per Topic')
plt.xticks(rotation=45, ha='right')
plt.legend(loc='upper left')
plt.grid(True)

plt.subplot(2, 1, 2)
sns.lineplot(x='time', y='comment', hue='topics', data=daily_comments, marker='o')
plt.xlabel('Date')
plt.ylabel('Number of Comments')
plt.title('Number of Comments Over Time per Topic')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Topic')
plt.grid(True)

plt.tight_layout()
plt.show()

df_heatmap = df_2.groupby(['topics', pd.Grouper(key='time', freq='W')])['sentiment_score'].mean().unstack()
plt.figure(figsize=(12, 6))
sns.heatmap(df_heatmap, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Average Weekly Sentiment by Topic")
plt.xlabel('Week')
plt.ylabel('Topic')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

quartile_thresholds = [-1, -0.25, 0, 0.25, 1]
quartile_labels = ['Very Negative', 'Negative', 'Positive', 'Very Positive']

df_2['sentiment_quartile'] = pd.cut(df_2['sentiment_score'], bins=quartile_thresholds, labels=quartile_labels, include_lowest=True)


quartile_counts = df_2.groupby(['topics', 'sentiment_quartile']).size().unstack(fill_value=0)


plt.figure(figsize=(12, 6))
sns.heatmap(quartile_counts, annot=True, fmt="d", cmap="viridis", linewidths=.5)
plt.xlabel('Sentiment Quartile')
plt.ylabel('Topic')
plt.title('Distribution of Sentiments across Topics')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

daily_sentiment = df_2.groupby(['topics', pd.Grouper(key='time', freq='D')])['sentiment_score'].mean().reset_index()

daily_comments = df_2.groupby(['topics', pd.Grouper(key='time', freq='D')])['comment'].count().reset_index()

x_axis_dates = daily_sentiment['time'].unique()

plt.figure(figsize=(15, 8))


for topic in daily_sentiment['topics'].unique():
    topic_data = daily_sentiment[daily_sentiment['topics'] == topic]
    y_values = topic_data['sentiment_score'].values

    x_values = [np.where(x_axis_dates == d)[0][0] for d in topic_data['time'].values]

    plt.plot(x_values, y_values, label=f'Topic {topic} Sentiment', linestyle='--')

for topic in daily_comments['topics'].unique():
    topic_data = daily_comments[daily_comments['topics'] == topic]
    y_values = topic_data['comment'].values

    x_values = [np.where(x_axis_dates == d)[0][0] for d in topic_data['time'].values]

    plt.plot(x_values, y_values / 10, label=f'Topic {topic} Comments (scaled)', linewidth=2)

peak_topic_data = daily_sentiment[daily_sentiment['topics'] == 0]
peak_x = np.argmax(peak_topic_data['sentiment_score'])
peak_y = peak_topic_data['sentiment_score'].max()
plt.annotate('Peak Topic 0', xy=(peak_x, peak_y), xytext=(peak_x + 2, peak_y + 0.2),
             arrowprops=dict(facecolor='black', shrink=0.05))


plt.title('Sentiment and Comment Analysis', fontsize=14)
plt.xlabel('Date', fontsize=10)
plt.ylabel('Sentiment Score / Comments (Scaled)', fontsize=10)
plt.xticks(ticks=range(len(x_axis_dates)), labels=x_axis_dates, rotation=45, ha='right')
plt.grid(True, linestyle='dotted', alpha=0.6)
plt.legend(loc='upper left', fontsize=8)
plt.show()

Congrats! You finished the notebook series of the Un4CHANate project! 